<a href="https://colab.research.google.com/github/mr-cri-spy/Movie_seat_enq_bot/blob/main/movie_seat_availability_agent_with_a_friendly_LLM_style_response_formatter_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# movie_api.py
from flask import Flask, request, jsonify

app = Flask(__name__)

# Simulated movie data
movie_data = {
    ("kingdom", "mysore"): {"seats_available": 47},
    ("vikram", "bangalore"): {"seats_available": 0},
}

@app.route('/movie')
def get_movie_seats():
    movie = request.args.get("movie", "").lower()
    city = request.args.get("city", "").lower()
    key = (movie, city)

    if key in movie_data:
        return jsonify({
            "status": "success",
            "seats_available": movie_data[key]["seats_available"]
        })
    else:
        return jsonify({
            "status": "error",
            "message": "No data found"
        })

if __name__ == "__main__":
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
python movie_api.py



In [ ]:


# Write the movie_bot_engine code to a file
with open("movie_bot_engine.py", "w") as f:
    f.write("""
import spacy
from bs4 import BeautifulSoup
import random

nlp = spacy.load("en_core_web_sm")

def extract_movie_city(text):
    doc = nlp(text)
    movie = city = None
    for ent in doc.ents:
        if ent.label_ == "GPE":
            city = city or ent.text
        elif ent.label_ in ["ORG", "WORK_OF_ART"]:
            movie = movie or ent.text
    return movie, city

def simulate_theatre_html(movie, city):
    theatres = ["Kingdome", "Inox Forum", "PVR Mall", "DRC Cinemas"]
    blocks = []
    for t in theatres:
        total = random.randint(80, 200)
        booked = random.randint(10, total - 20)
        available = total - booked
        html = f'''

            {movie} - {t}, {city}


        '''
        blocks.append(html)
    return "\n".join(blocks)

def parse_seats(html):
    soup = BeautifulSoup(html, "html.parser")
    theatres = soup.find_all("div", class_="theatre")
    parsed = []
    for t in theatres:
        name = t.find("h3").text
        seats = t.find("div", class_="seats")
        parsed.append({
            "theatre": name,
            "total": int(seats["data-total"]),
            "booked": int(seats["data-booked"]),
            "available": int(seats["data-available"])
        })
    return parsed
""")

In [ ]:
ngrok http 5000


In [ ]:
import requests

# Change these based on user input or chatbot parsing
movie = "Kingdom"
city = "Mysore"

# Replace with your ngrok link
ngrok_url = "https://abc123.ngrok.io"
endpoint = f"{ngrok_url}/movie?movie={movie}&city={city}"

# Generate LLM-style reply
def generate_response(movie, city, data):
    if data["status"] == "success" and data["seats_available"] > 0:
        return f" Yes! {data['seats_available']} seats are available for \"{movie}\" in {city}. Want to book?"
    elif data["status"] == "success" and data["seats_available"] == 0:
        return f" Sorry, no seats are currently available for \"{movie}\" in {city}."
    else:
        return f"⚠ Couldn't find any listings for \"{movie}\" in {city}. Please check the spelling or try a different movie."

# Call the API
try:
    res = requests.get(endpoint)
    if res.status_code == 200:
        response_data = res.json()
        final_answer = generate_response(movie, city, response_data)
    else:
        final_answer = " Server error. Try again later."
except Exception as e:
    final_answer = f" Request failed: {e}"

print(final_answer)
